# Solving a linear program in Python with Python-MIP

<b>Goals of this notebook:</b>
Learn some basic commands to solve an LP with Python.
These commands include `Model`, `add_var`, and `xsum`.

<b>Python packages required:</b>
mip

<b>Additional resources:</b>
The official Python-MIP documentation can be found at https://docs.python-mip.com/en/latest/index.html. See https://docs.python-mip.com/en/latest/examples.html for additional examples.


In its most general form, a linear program looks like

$$
\begin{array}{rcl}
    \max & c^\intercal x\\
    \text{s.t.}& Ax &\le& b\\
                     & Bx &=& d\\
                     & Cx &\ge& f\\
                     & x & \in & \mathbb{R}^n.
\end{array}
$$

There are three components to an LP: the variables $x$, the objective $\max ~ c^\intercal x$, and the constraints $Ax \le b$, $Bx = d$, and $Cx \ge f$.
In order to code these using Python, we can follow the following six steps:

<b>Step 1: Import Python's toolbox for solving LPs.</b> 
We will use the module called Python-MIP, which contains many useful linear programming tools.

<b>Step 2: Create an empty linear program.</b>
Intuitively, an empty linear program is Python's version of a sheet of paper on which we write the variables, the objetive, and the constraints.

<b>Step 3: Add the variables $x$.</b>
The three components of the LP involve $x$, so we need to create these first.

<b>Step 4: Add the objective</b> $\max ~ c^\intercal x$.

<b>Step 5: Add the constraints </b> $Ax \le b, ~Bx = d$, and $Cx \ge f$.

<b>Step 6: Solve the LP and print the results.</b>

The following example will help us introduce the basic Python commands needed in <b>Steps 1-6</b>.

### Example: Alice's Farm
Alice wants to build a farm to produce corn. She can spend CHF 1000.
It costs Alice CHF 3 to produce one kilogram of corn, and she can sell it for CHF 7.
Alice can also buy farmland at a cost of CHF 100 per acre, and each acre can only grow 30 kilograms of corn.
How many acres of farmland should she buy and how many kgs of corn should she produce to maximize profit?

Here is a model of Alice's problem.
    
$$
\begin{array}{rlcl}
\max & 7 \times (\text{corn produced})	\\
\text{s.t.} &  \text{corn produced} &\le& 30 \times (\text{acres purchased})\\
         & 3 \times (\text{corn produced}) + 100 \times (\text{acres purchased}) &\le& 1000\\
         &\text{corn produced} & \ge & 0\\
         &\text{acres purchased} & \ge &0.
\end{array}
$$

#### Step 1: Loading Python-MIP

Run the following line of code to import Python-MIP.

In [1]:
# Import the necessary LP toolbox from Python

from mip import *

<i>Note:</i> If the above gives an error, you should double-check the installation of the python-mip package. This is covered in the introduction to the conda and notebook environment.

#### Step 2: Creating an empty linear program

We create an empty linear program using the `Model` command as follows:

In [2]:
# Create an empty linear program

farm_lp = Model(name="Alice's_Farm", sense=MAXIMIZE)

The above line has the following components:

- `farm_lp` - This is the variable we use to store our LP. You can choose any name you want.

- `name="Alice's_Farm"` - This is the name displayed when we print the linear program. This is an optional parameter; you may choose any name you want or omit the `name` parameter.
   
- `sense=MAXIMIZE` - This makes the linear program a maximization problem. This is an optional parameter, so you may omit it, in which case the optimization sense will be set to _Minimize_ by default. Alternatively, we can set the sense by assigning it to `farm_lp.sense`, or at <b>Step 4</b> when adding the objective.
Note that we can simply write `MAXIMIZE` here because this is among the things that are imported from `mip`.

#### Step 3: Add the variables

Now we need to create two variables: one for the amount of corn produced and one for the number of acres purchased. 

We create a variable for the amount of corn produced using the `add_var` function of the LP class as follows:

In [3]:
# Create a variable for corn

corn = farm_lp.add_var(name="corn_produced", lb=0)

The above function call has the following two arguments:

- `name="corn_produced"` - This is the name of the variable that is displayed when we print the LP. You can choose any name you want. 

- `lb = 0` - This guarantees that the variable is lower bounded by 0 (i.e., that is it nonnegative). 

<i>Note:</i> The command `lb=0` creates the inequality $\text{corn} \ge 0$.
Alternatively, we can add this as a constraint in <b>Step 5</b>.
However, this type of constraint is so common that Python-MIP has commands to add it at this step.

The `add_var` function is used similarly to create a variable for the number of acres purchased.

In [4]:
# Create a variable for land

acres = farm_lp.add_var(name="acres_purchased", lb=0)

#### Step 4: Add the objective function

In <b>Step 2</b> that we created `farm_lp` to be a maximization problem. 
Therefore, we only need to add the objective function $c^\intercal x$, the optimization sense is already known to the LP.
The objective function for our problem is $7 \times \text{(corn produced)}$.
Using our variables from <b>Step 3</b>, the objective function becomes `7*corn`.
To add the objective to `farm_lp`, we save it in the attribute `objective` of `farm_lp`.

Run the following line of code to add the objective to `farm_lp`.

In [5]:
# Add the objective

farm_lp.objective = 7*corn

<i>Note:</i> If we would not have set the optimization sense to maximization in <b>Step 1</b>, we could now call `farm_lp.maximization(7*corn)` to set the objective and the LP sense at the same time.

#### Step 5: Add the constraints

The first constraint is $\text{corn produced} \le 30 \times (\text{acres purchased})$.
Using our variables this becomes `corn <= 30* acres`.
We use the command `+=` to add this to `farm_lp` as follows:

In [6]:
# Add the first constraint

farm_lp += corn <= 30*acres

The second constraint is $3 \times (\text{corn produced}) + 100 \times (\text{acres purchased}) \le 1000$.

Add this to our LP by running the following line of code.

In [7]:
# Add the second constraint

farm_lp += 3*corn +100*acres <= 1000

#### Step 6: Solve the LP and print the results

Now our LP is completely set up. Unfortunately, Python-MIP doesn't allow us to directly print out the LP to standard output (though it does allow us to export it to a file using the `write()` function described [here](https://python-mip.readthedocs.io/en/latest/classes.html#mip.Model.write)), but you can use the following function to double-check the model.

In [8]:
# This function creates a string representation of a given python-mip model
def model_to_str(model):
    s = f'{model.name}:\n{model.sense}\n{model.objective}\n'
    if model.constrs:
        s += 'SUBJECT TO\n' + '\n'.join(map(str, model.constrs)) + '\n'
    s += 'VARIABLES\n' + '\n'.join(f'{v.lb} <= {v.name} <= {v.ub} {v.var_type}' for v in model.vars)
    return s

# Display our LP
print(model_to_str(farm_lp))

Alice's_Farm:
MAX
+ 7.0corn_produced 
SUBJECT TO
constr(0): +1.0 corn_produced -30.0 acres_purchased <= -0.0
constr(1): +3.0 corn_produced +100.0 acres_purchased <= 1000.0
VARIABLES
0.0 <= corn_produced <= 1.7976931348623157e+308 C
0.0 <= acres_purchased <= 1.7976931348623157e+308 C


The following lines of code solves our linear program.

In [9]:
# Solve the linear program

farm_lp.optimize()

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 



<OptimizationStatus.OPTIMAL: 0>

Starting solution of the Linear programming problem using Primal Simplex

Coin0506I Presolve 0 (-2) rows, 0 (-2) columns and 0 (-4) elements
Clp0000I Optimal - objective value 1105.2632
Coin0511I After Postsolve, objective 1105.2632, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 1105.263158 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00


If everything went well, then the output should be `<OptimizationStatus.OPTIMAL: 0>`, which means that the LP was solved to optimality.

<i>Note: </i> There are several possible status values, including `OptimizationStatus.OPTIMAL`, `OptimizationStatus.INFEASIBLE`, and `OptimizationStatus.UNBOUNDED`. You can find the full list with descriptions at https://docs.python-mip.com/en/latest/classes.html#optimizationstatus.

We can check the optimization status not only by looking at the output of `optimize()`, but also by accessing the `status` attribute of our model. An example is provided below.

In [10]:
# Double-check the status of our LP

if farm_lp.status == OptimizationStatus.OPTIMAL:
    print('Our LP was solved to optimality.')
else:
    print(f'Something went wrong, the current status is {farm_lp.status}.')

Our LP was solved to optimality.


Now we (Alice and us) would like to know the optimal objective value and the optimal values of `corn` and `acres`.
The optimal value of `corn` can be accessed using `corn.x`.
Similarly, the optimal value of `acres` can be accessed using `acres.x`.
The optimal objective value of `farm_lp` is accessed using `farm_lp.objective_value`.

Below we access these attributes to display the optimal solution of our LP.

In [11]:
# Print the optimal value and the variables values

opt_corn = corn.x
print(f'Alice should produce {opt_corn:.2f} kilograms of corn.')

opt_acres = acres.x
print(f'Alice should purchase {opt_acres:.2f} acres of land.')

opt_val = farm_lp.objective_value
print(f'Alice will have a profit of CHF {opt_val:.2f}.')

Alice should produce 157.89 kilograms of corn.
Alice should purchase 5.26 acres of land.
Alice will have a profit of CHF 1105.26.


<i>Note:</i>
The character `%.2f` is a formatting tool for rounding a decimal to two places.

Congratulations! We have successfully solved our first LP. If everything was run correctly, then the output should be

    Alice should produce 157.89 kilograms of corn.
    Alice should purchase 5.26 acres of land.
    Alice will have a profit of CHF 1105.26.

#### Conclusions

There are six main steps to solving a linear program. 
Moreover, the basic commands that we learned are already enough to solve many optimization problems. You can see https://docs.python-mip.com/en/latest/examples.html for other examples (though they involve integer variables and are significantly more complex than what we covered here).

In the next example, we show how to use the `xsum` command in Python-MIP when dealing with multiple variables.

### Example: Bob's electric company

Alice's farm was a helpful example, but it was also simple with only two variables. 
What if we had a more complicated LP with more variables?
Some LPs can have thousands of variables!
In <b>Step 3</b> we could write the `add_var` command thousands of times on thousands of lines of code, but this would be tedious and prone to mistakes! 
Similarly, writing down one constraint that involves one thousand variables would be painful. 

In order to handle these complications, we can use the `xsum` function from Python-MIP. 
Let us illustrate this with another example.

Bob works for the electric company and must decide how to distribute electricity to the houses on his street. 
The street has seven houses, and each house requires at least 30 kWh per day. 
The company has two generators that can supply power to the houses.
Generator 1 can supply at most 100 kWh per day and generator 2 can supply at most 150 kWh.
The following price chart shows how much it costs (in CHF) each generator to supply 1 kWh to each house.

$$
\begin{array}{c|c|c|c|c|c|c|c}
&\text{House 1}&\text{House 2}&\text{House 3}&\text{House 4}&\text{House 5}&\text{House 6}&\text{House 7} \\
\hline
\text{Generator 1} & .17 & .25 & .29 & .16 & .24 & .20 & .29\\
\text{Generator 2} & .18 & .21 & .30 & .20 & .20 & .23 & .28\\
\end{array}
$$


How should Bob supply power to the houses in order to minimize his daily cost?

The following is a mathematical model of Bob's problem. 
We use a variable $x_{g,h}$ to denote the number of kWh that generator $g$ sends to house $h$.
For this model, we use the abbreviation $p_{g,h}$ to denote the price to send one kWh from generator $g$ to house $h$. 
For example $p_{2,6} = 0.23$. 

$$
\begin{array}{rlll}
\min & \displaystyle \sum_{g=1}^2 \sum_{h=1}^7 p_{g,h} \cdot x_{g,h}\\
\text{s.t.} & \displaystyle \sum_{h=1}^7 x_{1,h} \le 100 &\text{[Generator 1 can produce at most 100 kWh]}\\
&\displaystyle \sum_{h=1}^7 x_{2,h} \le 150 &\text{[Generator 2 can produce at most 150 kWh]}\\
& \displaystyle \sum_{g=1}^2 x_{g,h} \ge 30 & \text{for each } h =1,\dotsc, 7 &\text{[Each house needs at least 30 kWh]} \\
& x_{g,h} \ge 0 & \text{for each } g =1,2 \text{ and }h =1,\dotsc, 7.
\end{array}
$$

#### Solving the problem

Let us follow the steps outlined above. 
<b>Step 1</b> and <b>Step 2</b> load Python-MIP and create an empty LP.
Note that this is a minimization problem, so we use `MINIMIZE` in the function `Model`. (Alternatively, we could omit the `sense` parameter, as the optimization sense will be set to _Minimization_ by default.)
We will also store the prices in a 2-dimensional Python list.

Run the following line of code to run these steps. 

<i>Note: </i>
Lists are useful tools for storing information in Python, but we do not need to know how to manipulate them in this example.
All we need to know is that if we want to access a price, for instance the price from generator i to house j, then we use the code `priceGen[i-1][j-1]`.
Recall that Python lists begin their indexing at 0 and not 1 (for a more precise indexing system, consider using Python dictionaries).

In [12]:
# Import the necessary LP toolbox from Python

from mip import *

# Create an empty linear program

Electricity_LP = Model(name="Bob's_electricity", sense=MINIMIZE)

# Load the price chart
# We use the notation gxhy to indicate generator x and house y

pricesGen = [[0.17, 0.25, 0.29, 0.16, 0.24, 0.20, 0.29],
             [0.18, 0.21, 0.30, 0.20, 0.20, 0.23, 0.28]]

<b>Step 3</b> is to create the variables. 
There are 14 variables in total. As we said above, one way to add these variables to our model would be to define each variable on a new line by using lines of code like

`variable_1 = Electricity_LP.add_var(...)`.

However, a more convenient approach to defining these variables is to create the variables using list comprehension, which allows us to turn 14 lines of code into a few short lines.
We will take this second approach here:

In [13]:
# Create the variables

variables = [[Electricity_LP.add_var(name=f'g{gen}h{house}', lb=0)
              for house in range(7)]
              for gen in range(2)]

<i>Note: </i>
We can store the LP variables in a Python list, tuple, or dictionary. 
Here, we store the variables in a 2-dimensional list that is generated using Pyhton's list comprehension.

<b>Step 4</b> is to add the objective function $\sum_{g=1}^2 \sum_{h=1}^7 p_{g,h} \cdot x_{g,h}$. 
One approach to add the objective function is to write `Electricity_LP +=` followed by all 14 terms in the objective function.
However, as was the case when we added variables, this can be tedious and lead to mistakes.

Fortunately, Python-MIP has a function called `xsum` that allows us to create a linear expression from a list of terms. Moreover, we can use list comprehension to generate the terms and pass it to `xsum` in one line, like so:

In [14]:
# Add the objective

Electricity_LP += xsum(pricesGen[gen][house]*variables[gen][house] 
                       for gen in range(2) 
                       for house in range(7))

<b>Step 5</b> is to add the constraints. 
Here, we can use the `xsum` function to create the left hand side and then add the inequalities and the right hand sides (i.e., `<= 100`, `<= 150`, and `>= 30`).

Run the following code to add the constraints.

In [15]:
# Add the constraints

# The first set of constraints is for the generators

# Generator 1 can supply at most 100 kWh
Electricity_LP += xsum(variables[0]) <= 100

# Generator 2 can supply at most 150 kWh
Electricity_LP += xsum(variables[1]) <= 150

    
# The second set of constraints says each house must recieve at least 30 kWh

for house in range(7):
    Electricity_LP += xsum(variables[gen][house] for gen in range(2)) >= 30

Great! 
Now we have everything we need to solve our LP. 

In [16]:
# Display our linear program

print(model_to_str(Electricity_LP), '\n')

# Solve the linear program

Electricity_LP.optimize()

# Print the optimal value and the variables values

opt_val = Electricity_LP.objective_value
print(f'Bob must spend CHF {opt_val:.2f}.')

for gen in range(2):
    for house in range(7):
        opt_power = variables[gen][house].x
        print(f'Bob should send {opt_power:.2f} kWh from generator {gen+1} to house {house+1}.')

Bob's_electricity:
MIN
+ 0.17g0h0 + 0.25g0h1 + 0.29g0h2 + 0.16g0h3 + 0.24g0h4 + 0.2g0h5 + 0.29g0h6 + 0.18g1h0 + 0.21g1h1 + 0.3g1h2 + 0.2g1h3 + 0.2g1h4 + 0.23g1h5 + 0.28g1h6 
SUBJECT TO
constr(0): +1.0 g0h0 +1.0 g0h1 +1.0 g0h2 +1.0 g0h3 +1.0 g0h4 +1.0 g0h5 +1.0 g0h6 <= 100.0
constr(1): +1.0 g1h0 +1.0 g1h1 +1.0 g1h2 +1.0 g1h3 +1.0 g1h4 +1.0 g1h5 +1.0 g1h6 <= 150.0
constr(2): +1.0 g0h0 +1.0 g1h0 >= 30.0
constr(3): +1.0 g0h1 +1.0 g1h1 >= 30.0
constr(4): +1.0 g0h2 +1.0 g1h2 >= 30.0
constr(5): +1.0 g0h3 +1.0 g1h3 >= 30.0
constr(6): +1.0 g0h4 +1.0 g1h4 >= 30.0
constr(7): +1.0 g0h5 +1.0 g1h5 >= 30.0
constr(8): +1.0 g0h6 +1.0 g1h6 >= 30.0
VARIABLES
0.0 <= g0h0 <= 1.7976931348623157e+308 C
0.0 <= g0h1 <= 1.7976931348623157e+308 C
0.0 <= g0h2 <= 1.7976931348623157e+308 C
0.0 <= g0h3 <= 1.7976931348623157e+308 C
0.0 <= g0h4 <= 1.7976931348623157e+308 C
0.0 <= g0h5 <= 1.7976931348623157e+308 C
0.0 <= g0h6 <= 1.7976931348623157e+308 C
0.0 <= g1h0 <= 1.7976931348623157e+308 C
0.0 <= g1h1 <= 1.7976931

If everything ran correctly, then the output should say that Bob spends CHF 45.50.

#### Conclusions

A linear program can have multiple variables and constraints. 
There are certain tools in Python such as dictionaries and the `xsum` command to help us simplify our code when solving the LP.

### Try it yourself: A blending problem

A company produces cat food from several ingredients: chicken, beef, mutton, rice, wheat, and gel. The following table lists the cost (in CHF) and nutritional values (in g) of one gram of each ingredient, as well as nutritional standards that 100g of cat food has to satisfy.

| Ingredient              | Cost  | Protein | Fat    | Fibre  | Salt  |
| :---------------------- | :---- | :------ | :----- | :----- | :---- |
| Chicken                 | 0.012 | 0.100   | 0.060  | 0.001  | 0.002 |
| Beef                    | 0.013 | 0.150   | 0.100  | 0.050  | 0.005 |
| Mutton                  | 0.010 | 0.150   | 0.110  | 0.003  | 0.020 |
| Rice                    | 0.002 | 0.000   | 0.010  | 0.100  | 0.002 |
| Wheat bran              | 0.005 | 0.180   | 0.010  | 0.150  | 0.008 |
| Gel                     | 0.001 | 0.000   | 0.000  | 0.000  | 0.000 |
| Nutritional standard    | —     | ≥ 11    | ≥ 6    | ≤ 2    | ≤ 0.4 |

Your goal is to determine the composition of ingredients that minimizes the cost of the mixture while meeting the nutritional standards.

<b>Your task:</b> Solve the problem by creating a linear program whose optimal solution indicates an appropriate mixture, and solve the LP using Python-MIP.

In [ ]:
# Write your code here
costs = [0.012, 0.013, 0.01, 0.002, 0.005, 0.001]

nvals = [[0.1, 0.15, 0.15, 0, 0.17, 0], [0.06, 0.1, 0.11, 0.01, 0.01, 0], [0.001, 0.05, 0.003, 0.1, 0.15, 0], [0.002, 0.005, 0.02, 0.002, 0.008, 0]] # transposed view of the above table, easier to work with
ic = len(costs)

cat_lp = Model(name="Cat_Food", sense=MINIMIZE)
vars = [cat_lp.add_var(f"v{i}", lb = 0) for i in range(ic)] # one variable per ingredient

cat_lp += xsum(costs[i] * vars[i] for i in range(ic)) # objective, cost

cat_lp += xsum(nvals[0][i] * vars[i] for i in range(ic)) >= 11 # protein
cat_lp += xsum(nvals[1][i] * vars[i] for i in range(ic)) >= 6 # fat
cat_lp += xsum(nvals[2][i] * vars[i] for i in range(ic)) <= 2 # fibre
cat_lp += xsum(nvals[3][i] * vars[i] for i in range(ic)) <= 0.4 # salt

print(model_to_str(cat_lp), '\n')

cat_lp.optimize()

# Print the optimal value and the variables values

opt_val = cat_lp.objective_value
print(f'Minimum cost: {opt_val:.2f}.')

for var in vars:
    print(var.x)

Cat_Food:
MIN
+ 0.012v0 + 0.013v1 + 0.01v2 + 0.002v3 + 0.005v4 + 0.001v5 
SUBJECT TO
constr(0): +0.1 v0 +0.15 v1 +0.15 v2 +0.0 v3 +0.17 v4 +0.0 v5 >= 11.0
constr(1): +0.06 v0 +0.1 v1 +0.11 v2 +0.01 v3 +0.01 v4 +0.0 v5 >= 6.0
constr(2): +0.001 v0 +0.05 v1 +0.003 v2 +0.1 v3 +0.15 v4 +0.0 v5 <= 2.0
constr(3): +0.002 v0 +0.005 v1 +0.02 v2 +0.002 v3 +0.008 v4 +0.0 v5 <= 0.4
VARIABLES
0.0 <= v0 <= 1.7976931348623157e+308 C
0.0 <= v1 <= 1.7976931348623157e+308 C
0.0 <= v2 <= 1.7976931348623157e+308 C
0.0 <= v3 <= 1.7976931348623157e+308 C
0.0 <= v4 <= 1.7976931348623157e+308 C
0.0 <= v5 <= 1.7976931348623157e+308 C 

Starting solution of the Linear programming problem using Dual Simplex

Clp0024I Matrix will be packed to eliminate 5 small elements
Coin0506I Presolve 4 (0) rows, 5 (-1) columns and 19 (0) elements
Clp0006I 0  Obj 0 Primal inf 285.13673 (2)
Clp0000I Optimal - objective value 1.0704212
Coin0511I After Postsolve, objective 1.0704212, infeasibilities - dual 0 (0), primal 0 (0)
Clp0